<a href="https://colab.research.google.com/github/AnnSenina/Python_CL_2025/blob/main/%D0%9C%D0%B0%D0%BB%D0%B5%D0%BD%D1%8C%D0%BA%D0%BE%D0%B5_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_(%D0%BD%D0%B0_%D1%86%D0%B8%D1%84%D1%80%D0%B0%D1%85).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Начнем с очистки данных и дойдем до линейной регрессии

План примерно такой:

1. подготовим данные для анализа:
  - проверим типы данных, пропуски, выбросы
  - посмотрим готовую библиотеку, которая сделает это за вас
2. Найдем наиболее сильную линейную связь - построим самую простую модель регресии
3. Посмотрим в перспективу: как улучшать и как использовать


In [ ]:
# читаем данные, источник - https://www.kaggle.com/datasets/denkuznetz/food-delivery-time-prediction/data
import pandas as pd
delivery = pd.read_csv('https://github.com/AnnSenina/Other/raw/refs/heads/main/Food_Delivery_Times.csv')
delivery.head(3)

In [ ]:
df = delivery.copy()

In [ ]:
df.drop('Order_ID', axis=1, inplace=True)

In [ ]:
df.info()
# пропуски в Weather, Traffic_Level, Time_of_Day, Courier_Experience_yrs

In [ ]:
# отфильтруем или заполним мерами среднего
df['Weather'] = df['Weather'].fillna(df['Weather'].mode()[0])
df['Traffic_Level'] = df['Traffic_Level'].fillna(df['Traffic_Level'].mode()[0])
df['Time_of_Day'] = df['Time_of_Day'].fillna(df['Time_of_Day'].mode()[0])
df['Courier_Experience_yrs'] = df['Courier_Experience_yrs'].fillna(df['Courier_Experience_yrs'].median())
df.info()

In [ ]:
df.describe(include='all')
# Traffic_Level - порядкова шкала, можно перекодировать

In [ ]:
df['Traffic_Level'].value_counts()

In [ ]:
# Traffic_Level - порядковая шкала, можно перекодировать
df['Traffic_Level'] = df['Traffic_Level'].apply(lambda x: 0 if x == 'Low' else (1 if x == 'Medium' else 2))

### Стандартизация

Стандартизация наборов данных является общим требованием для многих моделей машинного обучения, реализованных в scikit-learn; отдельные признаки должны быть более или менее похожи на стандартные нормально распределенные данные: гауссовы с нулевым средним и единичной дисперсией

В sklearn есть несколько вариантов стандартизации. Один из самых распространенных и быстрых: StandardScalar: после масштабирования данные имеют нулевое среднее значение и единичную дисперсию

Другие: например, MinMaxScaler, normalize

[Документация](https://scikit-learn.ru/6-3-preprocessing-data/)

In [ ]:
# Посмотрим на одном столбце:
#!pip install -U scikit-learn

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df['Delivery_Time_min'] = scaler.fit_transform(df['Delivery_Time_min'].values.reshape(-1, 1))
df.head(3)

In [ ]:
print(round(df['Delivery_Time_min'].mean(), 0))
print(round(df['Delivery_Time_min'].std(), 0))

In [ ]:
# Подготовим все числовые столбцы к анализу: у нас есть минуты, километры, опыт курьера в годах
def scale_features(df):
    scaled = scaler.fit_transform(df)
    scaled = pd.DataFrame(scaled, columns=df.columns)
    return scaled

df_num = df.select_dtypes(include=['int64', 'float64'])

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
scale_features(df_num).boxplot()
plt.xticks(rotation=90);

In [ ]:
# применим функцию, которая посчитает выбросы в каждом столбеце:
def detect_outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = column[(column < lower_bound) | (column > upper_bound)]
    return len(outliers)

outliers_count = df_num.apply(detect_outliers)
outliers_count
# в идеале выбросы нужно отфильтровать

### Бонус

Есть библиотеки, которые сделают многое все за вас

In [ ]:
!pip install clean-df

In [ ]:
from clean_df import CleanDataFrame

cdf = CleanDataFrame(df=delivery)

In [ ]:
cdf.report()

### Подготовка завершена -> регрессия
(на самом деле, я не особо поработала с категориальными переменными, но сойдет)

In [ ]:
# библиотеки
import seaborn as sns
import scipy.stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np

In [ ]:
df_num.corr()

In [ ]:
plt.figure(figsize = (12,8))
sns.heatmap(df_num.corr(method='spearman'), annot = True, cmap='RdYlGn');

In [ ]:
plt.scatter(df_num['Delivery_Time_min'], df_num['Distance_km']);

In [ ]:
# функция линейной регрессии Y = a + bX

x = df_num['Distance_km']
y = df_num['Delivery_Time_min']
res = scipy.stats.linregress(x, y)
print(res)

print(round(res.rvalue**2, 2), 'коэффициент детерминации = 1 минус доля необъяснённой дисперсии')

In [ ]:
# функция линейной регрессии Y = a + bX

plt.plot(x, y, 'o', label='оригинальные данные')
plt.plot(x, res.intercept + res.slope*x, 'r', label='линия регрессии')
plt.legend();
# intercept - число, которое мы прибавляем в уравнении
# slope - коэффициент для х (а также наклон линии)

Способ получше

In [ ]:
x = df_num[['Distance_km']]
y = df_num['Delivery_Time_min']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)
# 70% данных - для обучения (train - тренировочные данные)
# 30% данных - для проверки, насколько модель точна (test - тестовые данные)
# random_state - восропроизводимость результата

In [ ]:
model = LinearRegression().fit(x_train, y_train)
y_pred = model.predict(x_test)
df_reg = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_reg

In [ ]:
# коэффициент детерминации
metrics.r2_score(y_test, y_pred)

А как улучшать дальше? (например, передать больше признаков)

In [ ]:
x = df_num[['Distance_km', 'Preparation_Time_min', 'Traffic_Level', 'Courier_Experience_yrs']]
y = df_num['Delivery_Time_min']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)
model = LinearRegression().fit(x_train, y_train)
y_pred = model.predict(x_test)

# точность модели должна вырасти
metrics.r2_score(y_test, y_pred)